# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Create Table

In [8]:
"""For table artist_song_for_session, the sessionId, itemInSession columns were used as 
   partition keys because the queries will filter by these columns in the WHERE clause."""

query = "CREATE TABLE IF NOT EXISTS artist_song_for_session "

query = query + "(sessionId     INT,   \
                  itemInSession INT,   \
                  artist        TEXT,  \
                  song          TEXT,  \
                  length        FLOAT, \
                  PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)
                    

### Insert Data Into Table

In [9]:
# STEP 2: Read event_datafile_new.csv that was created in the first section
# insert data into table artist_song_for_session created in STEP 1
# line [0] - artist
# line[1] - firstName
# line[2] - gender
# line[3] - itemInSession
# line[4] - lastName
# line[5] - length
# line[6] - level
# line[7] - location
# line[8] - sessionId
# line[9] - song
# line[10] - userId

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        
        query = "INSERT INTO artist_song_for_session \
                (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Run SELECT query to answer the question

In [10]:
# STEP 3: Run the query on our completed table to find out the song 
# where sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length from artist_song_for_session \
         WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Create Table

In [11]:
"""For table artist_song_user_for_user_session, the userId, sessionId columns were used as 
   partition keys because the queries will filter by these columns in the WHERE clause.
   The column itemInSession is used as the clustering key so that it can be used
   in the ORDER BY clause."""

query = "CREATE TABLE IF NOT EXISTS artist_song_user_for_user_session "
query = query + "(userId        INT,  \
                  sessionId     INT,  \
                  itemInSession INT,  \
                  artist        TEXT, \
                  song          TEXT, \
                  firstName     TEXT, \
                  lastName      TEXT, \
                  PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)



### Insert Data Into Table

In [12]:
# Insert data into table for query 2
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_song_user_for_user_session (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
                    

### Run SELECT query to answer the question

In [13]:
query = "SELECT artist, song, firstName, lastName \
         FROM artist_song_user_for_user_session \
         WHERE userId=10 AND sessionId=182 \
         ORDER BY itemInSession"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create Table

In [14]:
"""For table user_for_song, the song column was used as the partition key
   because the queries will filter by this column in the WHERE clause.
   The column userId is used as the clustering key so that the PRIMARY KEY
   is unique."""

query = "CREATE TABLE IF NOT EXISTS user_for_song "
query = query + "(firstName     TEXT, \
                  lastName      TEXT, \
                  song          TEXT, \
                  userId        INT,  \
                  PRIMARY KEY ((song), userId));"
session.execute(query)


### Insert Data Into Table

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        insert_query = """
        INSERT INTO user_for_song (firstName, lastName, userId, song)
        VALUES (%s, %s, %s, %s)
        """
        
        session.execute(insert_query, (line[1], line[4], int(line[10]), line[9]))

### Run SELECT query to answer the question

In [16]:
query = "SELECT firstName, lastName \
         FROM user_for_song \
         WHERE song='All Hands Against His Own' "

rows = session.execute(query)
for row in rows:
    print(row.firstname, row.lastname)        

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE IF EXISTS artist_song_for_session")
session.execute("DROP TABLE IF EXISTS artist_song_user_for_user_session")
session.execute("DROP TABLE IF EXISTS user_for_song")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()